[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_zugferd.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Create a ZUGFeRD invoice
Convert a PDF to PDF/A-3 and embed XML data to create a
ZUGFeRD-compliant invoice.

In [ ]:
import io
from pdftools_sdk.pdf import Document, Conformance
from pdftools_sdk.pdf_a.validation import Validator, AnalysisOptions
from pdftools_sdk.pdf_a.conversion import Converter, InvoiceType, EventSeverity, EventCategory, EventCode

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_path)
zugferd_xml_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/ZUGFeRD-invoice.xml'
zugferd_xml_path = 'ZUGFeRD-invoice.xml'
prepare_file(zugferd_xml_url, zugferd_xml_path)
output_path = 'ZUGFeRDInvoice.pdf'

In [ ]:
def add_zugferd_invoice(input_path: str, zugferd_xml_path: str, output_path: str):
    # Open input document
    with io.FileIO(input_path, 'rb') as input_stream:
        with Document.open(input_stream) as input_document:
            # Create the Validator object, and use the Conformance object to create
            # an AnalysisOptions object that controls the behavior of the Validator.
            validator = Validator()
            # The conformance has to be set to PDF/A-3 when adding the XML invoice file
            analysis_options = AnalysisOptions()
            analysis_options.conformance = Conformance.PDF_A3_U

            # Run the analysis
            analysis_result = validator.analyze(input_document, analysis_options)

            # Create a converter object
            converter = Converter()

            # Add the invoice XML file
            with io.FileIO(zugferd_xml_path, 'rb') as invoice_stream:
                converter.add_invoice_xml(InvoiceType.ZUGFERD, invoice_stream)

                # Add handler for conversion events
                event_severity_holder = [EventSeverity.INFORMATION]
                converter.add_conversion_event_handler(lambda *args: handle_conversion_event(*args, event_severity_holder))

                # Create output file
                with io.FileIO(output_path, 'wb+') as output_stream:
                    # Convert the input document to PDF/A
                    with converter.convert(analysis_result, input_document, output_stream) as output_document:
                        if event_severity_holder[0] == EventSeverity.INFORMATION:
                            print(f"Successfully converted document to {output_document.conformance.name}.")
                        elif event_severity_holder[0] == EventSeverity.WARNING:
                            print(f"Warnings occurred during the conversion to {output_document.conformance}.")
                            print("Check the output file to decide if the result is acceptable.")
                        elif event_severity_holder[0] == EventSeverity.ERROR:
                            raise Exception(f"Unable to convert document to PDF/A-3U because of critical conversion events.")

In [ ]:
def handle_conversion_event(data_part: str, message: str, severity: EventSeverity, category: EventCategory, code: EventCode, context: str, page_no: int, event_severity_holder: list[EventSeverity]):
    # Optionally the suggested severity can be changed according to
    # the requirements of your conversion process and, for example,
    # the event's category (e.Category).

    if severity > event_severity_holder[0]:
        event_severity_holder[0] = severity

    print(f"- {severity.name} {category.name}: {message} ({context}{f' on page {page_no}' if page_no > 0 else ''})")

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("INSERT-LICENSE-KEY")

    # Optional: Set your proxy configuration
    # Sdk.set_proxy("http://myproxy:8080")
    
    add_zugferd_invoice(input_path, zugferd_xml_path, output_path)

    print(f"Successfully created file {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")